<a href="https://colab.research.google.com/github/Partha-deuri/learn_pytorch/blob/main/webScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import requests
from bs4 import BeautifulSoup
import re

#initialize
globalCorpus = []
globalLinks = set()
globalVisited = set()

rootPageUrl = 'https://www.asomiyapratidin.in/'
globalLinks.add(rootPageUrl)

# helpers
skipLines = [
        '© asomiyapratidin 2024'
    ]
def printList(list):
    for item in list:
        print(item)
def removeRedundent(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    text = regrex_pattern.sub(r'',text)
    text  = re.sub("[a-zA-Z0-9#|/]",'',text)
    return text

# get all links

def fetchAllLinks (url,nested=10):
    if nested == 0:
        return
    req = requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")
    res=soup.find_all('a')
    for item in res:
        link = item.get('href')
        if link not in globalLinks:
            if re.search("^"+rootPageUrl+".*", link):
                globalLinks.add(link)
                fetchAllLinks(link,nested-1)

fetchAllLinks(rootPageUrl)


#get data from one page
def fetchData(link):
    corpus = []
    tempReq = requests.get(link)
    tempSoup = BeautifulSoup(tempReq.content,"html.parser")
    tempRes = tempSoup.findAll('p')
    for p in tempRes:
        if p.text not in skipLines:
            corpus.append(p.text)
    return corpus


# get data from all pages
def  getAllData(links=[]):
    for link in links:
        if link not in globalVisited:
            globalVisited.add(link)
            data = fetchData(link)
            if len(data) >0:
                globalCorpus.append(data)

getAllData(globalLinks)

# printList(globalCorpus)

# Create file
def createFile(filename):
    f = open(filename+".txt", "w", encoding='utf-8')
    for doc in globalCorpus:
        for para in doc:
            para = para.split('।')
            for line in para:
                # line = removeRedundent(line)
                line = re.sub(r" [^ঀ-ৱ]+",' ',line)
                f.write(line)
                f.write("\n")
    f.close()

createFile("assamese_corpus")

print("\n\n\t\t***** Task Completed *****\n\n")



		***** Task Completed *****




In [54]:
f = open("assamese_corpus.txt","r",encoding='utf-8')
print(f.read())
f.close()

ডিজিটেল সংবাদ জামুগুৰিহাটঃ জামুগুৰিহাটৰ বাঁপুজি ভৱনত আজি সদৌ জামুগুৰি ছাত্ৰ সন্থা আৰু উন্নতি সভাৰ যৌথ উদ্যোগত কৃতি ছাত্ৰ-ছাত্ৰীৰ এক বিশেষ অভিনন্দন অনুষ্ঠান অনুষ্ঠিত হয়৷ শোণিতপুৰ জিলা ছাত্ৰ সন্থাৰ সভাপতি অভিজিত নাথৰ উপস্থিতিত আয়োজিত বিশেষ অভিনন্দন অনুষ্ঠানটিত সদ্যঘোষিত উচ্চ আৰু উচ্চতৰ মাধ্যমিক চূড়ান্ত পৰীক্ষাত সুখ্যাতিৰে উত্তীৰ্ণ হৈ অঞ্চলটোলৈ গৌৰৱ কঢ়িয়াই অনা ৬৭০ গৰাকী কৃতি ছাত্ৰ-ছাত্ৰীক অভিনন্দন জনাই উজ্জ্বল ভৱিষ্যত কামনাৰে আশিৰ্বাদ প্ৰদান কৰা হয়৷
গাম্ভীৰ্যপূৰ্ণ অনুষ্ঠানটিৰ মুখ্য অতিথিৰ আসন অলংকৃত কৰি দিয়া ভাষণ প্ৰসংগত সদৌ অসম ছাত্ৰ সন্থাৰ সভাপতি উৎপল শৰ্ম্মাই কৃতি ছাত্ৰ-ছাত্ৰীসকলক অভিনন্দন জনাই উজ্জ্বল ভৱিষ্যত কামনাৰে সুদিহাপৰামৰ্শ আগবঢ়ায়৷ লগতে সদৌ অসম ছাত্ৰ সন্থাৰ সভাপতিগৰাকীয়ে প্ৰতিগৰাকী কৃতি ছাত্ৰ-ছাত্ৰীক একোখনকৈ অভিনন্দন পত্ৰ প্ৰদান কৰি ফুলাম গামোচাৰে আন্তৰিক শুভেচ্ছা জ্ঞাপন কৰে৷
ডিজিটেল ডেস্ক এফালে গ্ৰীষ্মপ্ৰবাহ, আনফালে খোৱাপানী এটুপিৰ বাবে হাহাকাৰ কৰিব লগা হৈছে একাংশ লোক
 দেশৰ ৰাজধানী দিল্লীত অৱ্যাহত আছে জলসংকট
 আজিও এটুপি খোৱাপানীৰ বাবে হাহাকাৰ কৰিব লগা হৈছে দিল্লীবাসী
 
ইফালে দ